## Data

In [23]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import shapely.geometry
import pyproj

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import re


def clean_conducta(conducta):
    conducta = conducta.upper()
    conducta = conducta.replace(" ", "_")
    conducta = re.sub(
        r"[ÁÉÍÓÚ]",
        lambda m: m.group(0)
        .replace("Á", "A")
        .replace("É", "E")
        .replace("Í", "I")
        .replace("Ó", "O")
        .replace("Ú", "U"),
        conducta,
    )
    conducta = conducta.replace("Ñ", "N")
    return conducta


crimes_df = pd.read_csv(
    "./base_crim16_08.csv"
)  # parse_dates=["fecha_hecho"], delimiter=";")

# Select relevant features
features = [
    "fecha_hecho",
    "fecha",
    "codigo_barrio",
    "year",
    "mes",
    "dia",
    "hora",
    "minuto",
    "latitud",
    "longitud",
    "conducta",
]
crimes_df = crimes_df[features]

crimes_df["fecha_hecho"] = pd.to_datetime(
    crimes_df["fecha_hecho"], format="%d/%m/%Y %H:%M"
)

print("crimes_df data shape ", crimes_df.shape)

# describe data and see if there are missing values
print(crimes_df.isnull().sum())  # check for missing values
crimes_df = crimes_df.dropna()  # drop missing values
crimes_df = crimes_df.drop_duplicates()  # elimianr los datos repetidos
# print("\ncrimes dtypes: ", crimes_df.dtypes)


# ? DATE TIME STAMP FUNCTION
column_1 = crimes_df.iloc[:, 0]

db = pd.DataFrame(
    {
        "dayofyear": column_1.dt.dayofyear,
        "week": column_1.dt.isocalendar().week,
        "weekofyear": column_1.dt.isocalendar().week,
        "dayofweek": column_1.dt.dayofweek,
        "quarter": column_1.dt.quarter,
        # ? kaggle
        "weekday": column_1.dt.weekday,
        "weekend": np.where(column_1.dt.weekday >= 4, 1, 0),
        # "Season": (
        #     column_1 - pd.DateOffset(months=1)
        # ).dt.quarter,
    }
)


crimes_df = crimes_df.drop("fecha_hecho", axis=1)  #! drop fecha_hecho
crimes_df = pd.concat([db, crimes_df], axis=1)
# data = pd.get_dummies(crimes_df.conducta)
# crimes_df = pd.concat([data, crimes_df], axis=1)
crimes_df["conducta"] = crimes_df["conducta"].apply(clean_conducta)

crimes_df = crimes_df[(crimes_df["year"] >= 2008) & (crimes_df["year"] <= 2018)]

crimes_df["codigo_barrio"] = crimes_df["codigo_barrio"].astype(str).str.replace("#", "")

# parsear el dato de codigo_barrio a int, los que no se puedan parsear a interos se eliminan esos registros del dataframe
crimes_df["codigo_barrio"] = pd.to_numeric(crimes_df["codigo_barrio"], errors="coerce")

crimes_df.head(10)

crimes_df data shape  (377282, 11)
fecha_hecho          0
fecha                0
codigo_barrio        0
year                 0
mes                  0
dia                  0
hora                 0
minuto               0
latitud          24239
longitud         24239
conducta             0
dtype: int64


,dayofyear,week,weekofyear,dayofweek,quarter,weekday,weekend,fecha,codigo_barrio,year,mes,dia,hora,minuto,latitud,longitud,conducta
0,2,1,1,0,1,0,0,2/01/2017,1006,2017,1,2,17,30,6.251426,-75.570280,HURTO_DE_CARRO
1,2,1,1,0,1,0,0,2/01/2017,519,2017,1,2,18,0,6.274359,-75.578320,HURTO_DE_CARRO
2,2,1,1,0,1,0,0,2/01/2017,710,2017,1,2,15,30,6.283381,-75.580012,HURTO_DE_CARRO
3,3,1,1,1,1,1,0,3/01/2017,809,2017,1,3,0,15,6.246295,-75.552469,HURTO_DE_CARRO
4,3,1,1,1,1,1,0,3/01/2017,1004,2017,1,3,15,0,6.262067,-75.570167,HURTO_DE_CARRO
5,3,1,1,1,1,1,0,3/01/2017,1016,2017,1,3,3,30,6.247649,-75.557575,HURTO_DE_CARRO
6,4,1,1,2,1,2,0,4/01/2017,504,2017,1,4,22,0,6.306256,-75.566786,HURTO_DE_CARRO
7,5,1,1,3,1,3,0,5/01/2017,1107,2017,1,5,21,0,6.247196,-75.587879,HURTO_DE_CARRO
8,5,1,1,3,1,3,0,5/01/2017,1609,2017,1,5,22,30,6.203295,-75.598008,HURTO_DE_CARRO
9,5,1,1,3,1,3,0,5/01/2017,908,2017,1,5,19,30,6.236215,-75.549798,HURTO_DE_CARRO


### SHP and intersections

#### Grids

In [ ]:
# Set up transformers, EPSG:3857 is metric, same as EPSG:900913
# to_proxy_transformer = pyproj.Transformer.from_crs("epsg:4326", "epsg:32618")
# to_original_transformer = pyproj.Transformer.from_crs("epsg:4326", "epsg:32618")


# # Definir la función de conversión de grados a metros
# def convertir_lat_long_a_metros(lat, long):
#     transformed = to_proxy_transformer.transform(lat, long)
#     return transformed[0], transformed[1]


# # Perform spatial join by rounding coordinates for more accurate intersection
# # def round_coords(geom):
# #     if geom.type == "Polygon":
# #         return shapely.geometry.Polygon(
# #             [(round(x, 4), round(y, 4)) for x, y in geom.exterior.coords]
# #         )
# #     elif geom.type == "Point":
# #         return Point(round(geom.x, 4), round(geom.y, 4))
# #     else:
# #         return geom


# def create_gdf(df):
#     gdf = df.copy()
#     gdf["Coordinates"] = list(zip(gdf.longitud, gdf.latitud))
#     gdf.Coordinates = gdf.Coordinates.apply(Point)
#     gdf = gpd.GeoDataFrame(gdf, geometry="Coordinates", crs="epsg:32618")
#     return gdf


# # Leer el archivo SHP y crear un objeto GeoDataFrame
# ruta_archivo_shp = "shp/BarrioVereda_2014.shp"
# #ruta_archivo_shp = "shp/map.shp"
# gdf = gpd.read_file(ruta_archivo_shp)

# # Filtrar por el código de comunas de Medellín (son 16 comunas en total)
# gdf = gdf[gdf["LIMITECOMU"] < "17"]

# # Reproyectar a coordenadas planas
# gdf = gdf.to_crs(epsg=32618)

# print(gdf.columns.tolist()) #mostar los nombres de las columnas de la base

# # Crear el polígono único que representa el límite exterior de todos los polígonos
# outter_map = gdf.unary_union

# # Crear un nuevo GeoDataFrame con el polígono del límite exterior
# gdf_outter = gpd.GeoDataFrame(geometry=[outter_map], crs=gdf.crs)

# xmin, ymin, xmax, ymax = gdf_outter.total_bounds
# print("xmin:", xmin, "ymin:", ymin, "xmax:", xmax, "ymax:", ymax)

# stepsize = 200  # 200 meters grid step size

# # Iterate over 2D area and create grid squares (boxes) with consecutive grid_id
# gridboxes = []
# grid_id = 1  # Initialize grid_id to start from 1
# x = xmin
# while x < xmax:
#     y = ymin
#     while y < ymax:
#         box = shapely.geometry.box(x, y, x + stepsize, y + stepsize)
#         box_properties = {"geometry": box, "grid_id": grid_id}
#         gridboxes.append(box_properties)
#         grid_id += 1  # Increment grid_id for the next box
#         y += stepsize
#     x += stepsize

# # Create a GeoDataFrame from gridboxes
# grid_gdf = gpd.GeoDataFrame(gridboxes, crs=gdf.crs)
# # grid_gdf["geometry"] = grid_gdf["geometry"].apply(round_coords)

# print(grid_gdf.columns.tolist()) #mostar los nombres de las columnas de la base AQUI SE MUESTRA AREA

# # Perform spatial join
# grid_outter_gdf = gpd.sjoin(grid_gdf, gdf_outter, how="inner", op="within")
# # grid_outter_gdf["geometry"] = grid_outter_gdf["geometry"].apply(round_coords)

# print(grid_outter_gdf.columns.tolist()) #mostar los nombres de las columnas de la base AQUI SE MUESTRA AREA

# # **** 2. Cargar los datos de los delitos ****
# crimes_gdf = create_gdf(crimes_df)
# print(len(crimes_gdf))
# # crimes_gdf["Coordinates"] = crimes_gdf["Coordinates"].apply(round_coords)

# # Convertir las coordenadas de latitud y longitud a metros y crear una nueva columna
# crimes_gdf["Coordinates"] = crimes_gdf.apply(
#     lambda row: Point(*convertir_lat_long_a_metros(row["latitud"], row["longitud"])),
#     axis=1,
# )


# # eliminar todos los coordinates que tengan infinity
# crimes_gdf = crimes_gdf[
#     crimes_gdf["Coordinates"].apply(lambda x: not np.isinf(x.coords[0][0]))
# ]

# # eliminar todos los registros que tengan latitud y longitud 0
# crimes_gdf = crimes_gdf[
#     crimes_gdf["Coordinates"].apply(
#         lambda x: x.coords[0][0] != 0 and x.coords[0][1] != 0
#     )
# ]

# # Realizar la intersección usando gpd.sjoin()
# crimes_gdf = gpd.sjoin(
#     crimes_gdf, grid_outter_gdf[["grid_id", "geometry"]], how="inner", op="within"
# )
# print(len(crimes_gdf))

# # eliminar columna de index_right
# crimes_gdf = crimes_gdf.drop(columns=["index_right"])

# print(crimes_gdf.head())

# print("\n --------------------")

# print(grid_outter_gdf.head())

# # Plot the map and grid squares
# fig, ax = plt.subplots(figsize=(10, 8))
# grid_outter_gdf.plot(ax=ax, color="white", edgecolor="black")
# crimes_gdf.plot(ax=ax, color="red", markersize=1)
# # gdf_outter.plot(ax=ax, color='lightgray', edgecolor='black')


# plt.title("Mapa con cuadrícula de cuadrados de 200 metros")
# plt.xlabel("Longitud (EPSG:32618)")
# plt.ylabel("Latitud (EPSG:32618)")
# plt.show()
#print(gdf.columns.tolist()) #para ver el nombre de las columnas de una base de datos

### Barrios

In [40]:
# Set up transformers, EPSG:3857 is metric, same as EPSG:900913
to_proxy_transformer = pyproj.Transformer.from_crs("epsg:4326", "epsg:32618")
to_original_transformer = pyproj.Transformer.from_crs("epsg:4326", "epsg:32618")

# Definir la función de conversión de grados a metros
def convertir_lat_long_a_metros(lat, long):
    transformed = to_proxy_transformer.transform(lat, long)
    return transformed[0], transformed[1]

def create_gdf(df):
    gdf = df.copy()
    gdf["Coordinates"] = list(zip(gdf.longitud, gdf.latitud))
    gdf.Coordinates = gdf.Coordinates.apply(Point)
    gdf = gpd.GeoDataFrame(gdf, geometry="Coordinates", crs="epsg:32618")
    return gdf

crimes_gdf = create_gdf(crimes_df)
# print("len crimes_gdf ->",len(crimes_gdf))

# Convertir las coordenadas de latitud y longitud a metros y crear una nueva columna
crimes_gdf["Coordinates"] = crimes_gdf.apply(
    lambda row: Point(*convertir_lat_long_a_metros(row["latitud"], row["longitud"])),
    axis=1,
)

# eliminar todos los coordinates que tengan infinity
crimes_gdf = crimes_gdf[
    crimes_gdf["Coordinates"].apply(lambda x: not np.isinf(x.coords[0][0]))
]

# eliminar todos los registros que tengan latitud y longitud 0
crimes_gdf = crimes_gdf[
    crimes_gdf["Coordinates"].apply(
        lambda x: x.coords[0][0] != 0 and x.coords[0][1] != 0
    )
]

# ***************** Leer el archivo SHP y crear un objeto GeoDataFrame ***************
ruta_shp = "shp/map.shp"
gdf = gpd.read_file(ruta_shp)

# Filtrar por el código de comunas de Medellín (son 16 comunas en total)
gdf = gdf[gdf["LIMITECOMU"] < "17"]

# Reproyectar a coordenadas planas
gdf = gdf.to_crs(epsg=32618)

# imprimir algunas filas del archivo shp
print(gdf.head(5))

# Realizar la intersección usando gpd.sjoin()
crimes_gdf = gpd.sjoin(
    crimes_gdf, gdf[["area","geometry"]], how="inner", op="within"
)

# eliminar columna de index_right
crimes_gdf = crimes_gdf.drop(columns=["index_right"])

print("\n ---------------------------------------------------------- \n")
# imprimer algunos datos aleatorios del dataframe crimes_gdf
print(crimes_gdf.sample(5))

    CODIGO                  NOMBRE  ABREVIATUR IDENTIFICA LIMITECOMU  \
0  Inst_15                   U.P.B         NaN    Inst_15         11   
1  Inst_18          Cerro Nutibara         NaN    Inst_18         16   
2     0610        Mirador del Doce         NaN       0610         06   
3     1620  El Nogal-Los Almendros         NaN       1620         16   
4     0612              El Triunfo         NaN       0612         06   

   ZHFISICAGE LIMITEMUNI  VIGENCIA_F  SUBTIPO_BA  LINK_DOCUM           area  \
0         NaN        001         NaN         1.0         NaN  216275.239275   
1         NaN        001         NaN         1.0         NaN  398713.874885   
2         NaN        001         NaN         1.0         NaN   64749.995150   
3         NaN        001         NaN         1.0         NaN  248088.656801   
4         NaN        001         NaN         1.0         NaN  116938.310530   

                                            geometry  
0  POLYGON ((434831.540 690264.857, 4

c:\Users\Nicolas\Documents\TRABAJOS\TESIS_CAMILA\workspace_python\myvenv39\lib\site-packages\IPython\core\interactiveshell.py:3448: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):



 ---------------------------------------------------------- 

        dayofyear  week  weekofyear  dayofweek  quarter  weekday  weekend  \
73816          87    13          13          0        1        0        0   
87612         182    27          27          1        3        1        0   
19922         219    32          32          2        3        2        0   
98361         229    34          34          0        3        0        0   
333216        124    18          18          6        2        6        1   

             fecha  codigo_barrio  year  mes  dia  hora  minuto   latitud  \
73816   28/03/2011           1001  2011    3   28     0       5  6.263111   
87612   1/07/2014             101  2014    7    1    18      30  6.295385   
19922   6/08/2008            1108  2008    8    6    19       0  6.238979   
98361   17/08/2015           1006  2015    8   17     6       0  6.253477   
333216  4/05/2014             803  2014    5    4     4      27  6.255785   

         lo

In [41]:
# Eliminar espacios al principio y al final de los valores
crimes_gdf["fecha"] = crimes_gdf["fecha"].str.strip()

# Convertir a datetime
crimes_gdf["fecha"] = pd.to_datetime(crimes_gdf["fecha"], format="%d/%m/%Y")

# Agrupar los datos por 'codigo_barrio', 'fecha' y 'conducta' y contar las ocurrencias
crime_count_per_day = crimes_gdf.groupby(
    ["codigo_barrio", "fecha", "conducta"]
).size()  # ,'year'

# Desapilar y rellenar los datos faltantes con 0
crime_count_per_day = (
    crime_count_per_day.unstack(["codigo_barrio", "conducta"])
    .asfreq("D")
    .fillna(0)
    .stack(["codigo_barrio", "conducta"])
    .reset_index(name="crime_count")
)

# Renombrar la columna de conteo
crime_count_per_day = pd.DataFrame(crime_count_per_day)

crime_count_per_day.sample(5)

# crime_count_per_day.to_csv('crimecount.csv')

,fecha,codigo_barrio,conducta,crime_count
3426605,2015-09-07,724,EXTORSION,0.0
2332467,2013-03-25,608,EXTORSION,0.0
3523723,2015-11-25,1605,HURTO_A_PERSONA,0.0
3858476,2016-08-26,301,HURTO_DE_CARRO,0.0
3623367,2016-02-15,1001,HURTO_DE_CARRO,0.0


### Load and Merge characteristics

In [ ]:
characteristics = pd.read_excel("./characteristics_edad.xlsx")

characteristics = characteristics.drop(
    [
        "rentista",
        "viv_desechos",
        "otra_acti",
        "union_libre",
        "apto",
        "casa",
        "especial",
        "cuartos",
        "bloque",
        "divor",
        "viudo",
        "mate_pre",
        "inca_work",
        "ladrillo",
        "si_afili_pen",
        "si_arl",
        "cemento",
        "madera_burda",
        "mate_desechos",
        "baldosa",
        "tierra_arena",
        "si_estu",
        "madera_b",
    ],
    axis=1,
)
# eliminaar registros que tengan CODIGO null nan o con un caracter '.'
characteristics = characteristics[characteristics["CODIGO"].notna()]
characteristics = characteristics[characteristics["CODIGO"] != "."]

# convertir a entero el codigo
characteristics["CODIGO"] = characteristics["CODIGO"].astype(int)

# renombrar columna de characteristics "CODIGO" a "codigo_barrio"
characteristics = characteristics.rename(columns={"CODIGO": "codigo_barrio"})

# Se realiza el merge de crime_count_per_day characteristics left de crime_count_per_day
crime_count_per_day['year'] = pd.to_datetime(crime_count_per_day['fecha']).dt.year
crime_count_per_day = crime_count_per_day.merge(characteristics, how='left', on=['codigo_barrio', 'year'])
crime_count_per_day.sample(10)

In [ ]:
# Description of our new dataset 
crime_count_per_day.describe()

In [ ]:
print("Missing values: ")
print(crime_count_per_day.isnull().sum())
# drop missing values
print("dropping missing values")
crime_count_per_day = crime_count_per_day.dropna()
# elimianr los datos repetidos
print("eliminar los datos repetidos")
crime_count_per_day = crime_count_per_day.drop_duplicates()

#crime_count_per_day.to_csv('crimecount_merge.csv')
crime_count_per_day.sample(10)

In [ ]:
crime_count_per_day.dtypes

In [ ]:
#Instances of Crime_count per date, neighborhood, and crime category
cc = crime_count_per_day.groupby('Crime_count').size()
fig, axes = plt.subplots(1, 2, figsize=(14, 4));
cc.plot.bar(logy=False, ax=axes[0], title='Instances of Crime_count, lin axes');
cc.plot.bar(logy=True, ax=axes[1], title='Instances of Crime_count, log-lin axes');

### Correlation features

In [ ]:
# pip install seaborn

In [ ]:
# import seaborn as sns
# sns.set()
# #Adding dummy feature to get full cmap scale (really complicated to solve in sns/matplotlib)
# crime_count_per_day['dummy_feature'] = -crime_count_per_day['pensionado']
# corr = crime_count_per_day.corr()

# # Create a heatmap
# fig, ax = plt.subplots(figsize=(10, 8))
# sns.heatmap(corr,
#             xticklabels=corr.columns.values,
#             yticklabels=corr.columns.values,
#             cmap='RdBu_r',
#             ax=ax)

# # Remove the dummy feature
# crime_count_per_day = crime_count_per_day.drop('dummy_feature', axis=1)

# plt.show()

### Train-dev-test split

In [ ]:
# Sort columns on name for easier handling of predictions downstream
crime_data_grd = crime_count_per_day.reindex(
    sorted(crime_count_per_day.columns), axis=1
)

X_train = crime_data_grd[crime_data_grd['year'] <= 2017] 
#crime_data_grd.query('fecha < "2018-01-01"')
X_test = crime_data_grd[crime_data_grd['year'] == 2018]  
#crime_data_grd.query('fecha >= "2018-01-01"')

#labels for classifier
y_train_clf = X_train['Crime_count'].clip(upper=1).astype(int)
y_test_clf = X_test['Crime_count'].clip(upper=1).astype(int)

#labels for regression
y_train_reg = X_train['Crime_count']
y_test_reg = X_test['Crime_count']

#Clean
X_train = X_train.drop(['Crime_count'],axis=1)
X_test = X_test.drop(['Crime_count'],axis=1)

#Check split ratios
time_split = int(len(X_train)/(len(crime_data_grd))*100)
event_split = int(y_train_clf.sum()/(y_train_clf.sum()+y_test_clf.sum())*100)
print("Train/test split in time: {}/{}".format(time_split, 100-time_split))
print("Train/test split in events: {}/{}".format(event_split, 100-event_split))

In [ ]:
X_train.head()

### Model preprocessing pipeline

In [ ]:
pip install sklearn-pandas

In [ ]:
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelBinarizer
from sklearn_pandas import DataFrameMapper, gen_features
from sklearn.pipeline import FeatureUnion

#source: https://github.com/ageron/handson-ml
class LabelBinarizerPipelineFriendly(LabelBinarizer):
    """Binarize Label in a One vs all fashion. This utility class would
    enable people to use this with categorical variables in pipelines.
    Refer to LabelBinarizer for detailed info.
    """

    def fit(self, X, y=None):
        """this would allow us to fit the model based on the X input."""
        super(LabelBinarizerPipelineFriendly, self).fit(X)

    def transform(self, X, y=None):
        return super(LabelBinarizerPipelineFriendly, self).transform(X)

    def fit_transform(self, X, y=None):
        return super(LabelBinarizerPipelineFriendly, self).fit(X).transform(X)


In [ ]:
#Numerical features. Using a StandardScaler to be insensitive to any outliers.
num_attributes = X_train.select_dtypes(exclude=[object]).columns
num_mapper = DataFrameMapper(gen_features(
    columns=[[f] for f in num_attributes],
    classes=[
            StandardScaler
            ]
))

#Categorical features. One-hot encode (binarize) the features.
cat_attributes = X_train.select_dtypes(include=[object]).columns
cat_mapper = DataFrameMapper(gen_features(
    columns=[[f] for f in cat_attributes],
    classes=[LabelBinarizerPipelineFriendly]
))

#Union
full_pipeline = FeatureUnion(transformer_list=[
    ('num_mapper', num_mapper),
    ('cat_mapper', cat_mapper),
])

X_train_prep = full_pipeline.fit_transform(X_train)
X_test_prep = full_pipeline.transform(X_test)
print(num_attributes)
print(cat_attributes)
print(np.shape(X_train_prep))
print(X_train_prep[1])


### Hyperparameter grid search

In [ ]:
from scipy.stats import randint as sp_randint
from sklearn.metrics import precision_recall_curve, confusion_matrix, make_scorer, classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_predict, RandomizedSearchCV
# def tp(y_true, y_pred): return confusion_matrix(y_true, y_pred)[1, 1]
# def tn(y_true, y_pred): return confusion_matrix(y_true, y_pred)[0, 0]
# def fp(y_true, y_pred): return confusion_matrix(y_true, y_pred)[0, 1]
# def fn(y_true, y_pred): return confusion_matrix(y_true, y_pred)[1, 0]
# def get_standard_scores():
#     scores = {'recall' : 'recall', 'precision' : 'precision',
#     'roc_auc' : 'roc_auc', 'avg_precision' : 'average_precision',
#     'f1' : 'f1', 
#     'tp' : make_scorer(tp), 'tn' : make_scorer(tn),
#     'fp' : make_scorer(fp), 'fn' : make_scorer(fn)}
#     return scores
# #Uncomment the three bottom lines in the cell and it'll search and print the results.
# param_grid = {
#     'max_depth': sp_randint(5, 80),
#     'n_estimators' : sp_randint(1, 200),
#     'min_samples_leaf' : sp_randint(1, 400),
#     'random_state' : [42],
#     'max_features' : ['auto'],
#     'class_weight' : ['balanced', None]
#     }
# def generate_results_df(grid_searcher, scores, generate_files=False):
#     #keep all columns for future analyses
#     result_df_full = pd.DataFrame(grid_searcher.cv_results_)
#     #create a nimbler df
#     result_df_compact = pd.DataFrame(index=result_df_full.index)
#     for col in [col for col in result_df_full.columns if 'mean_test' in col]:
#         #First, check to see if we have a mean_test_score column, whcih indicates only one unnamed metric (from BayesSearchCV)
#         #If so, fill in name; else, handle multiple metrics. We don't have to break the loop, since mean_test_score implies only one 'mean_test'-column.
#         if 'mean_test_score' in col and type(scores) is str:
#             result_df_compact[scores] = result_df_full[[col, col.replace('mean_', 'std_')]].round(3).astype(str).apply(lambda x: ' +/- '.join(x), axis=1)
#         else:
#             #Confusion matrix must be handled separately
#             if '_tp' not in col and '_tn' not in col and '_fn' not in col and '_fp' not in col:
#                 result_df_compact[col.replace('mean_test_','')] = result_df_full[[col, col.replace('mean_', 'std_')]].round(3).astype(str).apply(lambda x: ' +/- '.join(x), axis=1)
#             else:
#                 #only do the confusion once
#                 if 'tp_fp_fn_tn' not in result_df_compact.columns:
#                     for metric in ['tp','fp','fn','tn']:
#                         #sum confusion matrix elements to match total number of samples
#                         result_df_full[metric+'_sum'] = result_df_full[[column for column in result_df_full.columns if metric in column and 'split' in column]].sum(axis=1)
#                     result_df_compact['tp_fp_fn_tn'] = result_df_full[['tp_sum', 'fp_sum', 'fn_sum', 'tn_sum']].astype(int).astype(str).apply(lambda x: '/'.join(x), axis=1)
#     result_df_compact['time_fit_score'] = result_df_full[['mean_fit_time', 'mean_score_time']].round(3).astype(str).apply(lambda x: ' + '.join(x), axis=1)
#     #add all parameters individually to the compact df
#     param_cols=[col for col in result_df_full.columns if 'param_' in col]
#     #transfer all parameter columns to compact df
#     for col in param_cols:
#         result_df_compact[col] = result_df_full[col].copy()
#     #we usually want to save the results in a file
#     if generate_files:
#         estimator=str(grid_searcher.estimator).split('(')[0]+'_'
#         #If we use a pipeline, list all estimators/samplers/tranformers in the file name
#         if estimator=='Pipeline_':
#             estimator=""
#             for key, value in grid_searcher.estimator.named_steps.items():
#                 estimator+=str(value).split('(')[0]+'_'
#         #TODO: set output dir of file
#         opwd = './'
#         ofile = 'Results_df_'+estimator+time.strftime("%Y%m%d_%H%M%S")+'.csv'
#         result_df_compact.to_csv(opwd+ofile, sep=',')
#         ofile = 'Full_results_df_'+estimator+'_'+time.strftime("%Y%m%d_%H%M%S")+'.csv'
#         result_df_full.to_csv(opwd+ofile, sep=',')
#     return result_df_full, result_df_compact

# scores = get_standard_scores()

# n_iter = 10; cv=3
# rfc = RandomForestClassifier()
# rnd_search = RandomizedSearchCV(rfc, param_grid, n_iter=n_iter, cv=cv, scoring=scores, n_jobs=-1, refit=False, return_train_score=False, verbose=2)
# rnd_search.fit(X_train_prep, y_train_clf)

# _, gs_df = generate_results_df(rnd_search, scores)
# gs_df.sort_values(by='avg_precision', ascending=False)

In [ ]:
#Select best models: best balanced classes and unbalanced classes. 
#Could use refit=True in search above if you want to include search and run code top-to-bottom.
rfc1 = RandomForestClassifier(random_state=42, n_jobs=-1, class_weight=None, max_depth=77, n_estimators=81, min_samples_leaf=152)
rfc1.fit(X_train_prep, y_train_clf)
#rfc2 = RandomForestClassifier(random_state=42, n_jobs=-1, class_weight='balanced', max_depth=69, n_estimators=64, min_samples_leaf=132)
#rfc2.fit(X_train_prep, y_train_clf)

### Model evaluation

In [ ]:
def obtain_and_plot_evaluation_figures(model, X, y, method, datatype, cv=3, verbose=100, title_prefix=None):
    if datatype == 'train':
        y_pred = cross_val_predict(model, X, y, cv=cv, method=method, verbose=verbose)
    elif datatype == 'test':
        y_pred = model.predict_proba(X)
    precisions, recalls, thresholds = precision_recall_curve(y, y_pred[:,1])
        #y_pred = model.predict_proba(X)
        
    # if datatype == 'train':
    #     precisions, recalls, thresholds = precision_recall_curve(y, y_pred)
    # else:
    #     precisions, recalls, thresholds = precision_recall_curve(y, y_pred)

    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(15,5));
    ax1.plot(thresholds, precisions[:-1], "b--", label="Precision");
    ax1.plot(thresholds, recalls[:-1], "g-", label="Recall");
    ax1.set(xlabel="Thresholds");
    ax1.set_ylim([0,1]);
    ax1.legend();
    ax1.set_title(title_prefix + 'Precision & recall at thresholds');

    ax2.plot(precisions, recalls, label=None);
    ax2.set_ylim([0,1]);
    ax2.set_xlim([0,1]);
    ax2.set(xlabel="Precision", ylabel="Recall");
    ax2.set_title(title_prefix + 'Precision-recall curve');
    print(np.shape(y_pred))

In [ ]:
pip install joblib


In [ ]:

import joblib
# Guardar el modelo en un archivo con compresión
model_filename = 'random_forest_model_compre_16_08.joblib'
joblib.dump(rfc1, model_filename, compress=5)  # El valor 3 controla el nivel de compresión
# Cargar el modelo desde el archivo
loaded_model = joblib.load(model_filename)

In [ ]:
print(y_train_clf.head())

print(y_train_clf.shape)

In [ ]:
obtain_and_plot_evaluation_figures(model=rfc1, X=X_train_prep, y=y_train_clf, datatype='train', method='predict_proba', cv=3, verbose=0, title_prefix='CV:ed TRAINING DATA: ')

In [ ]:
obtain_and_plot_evaluation_figures(model=rfc1, X=X_test_prep, y=y_test_clf, datatype='test', method='predict_proba', verbose=0, title_prefix='TEST DATA: ')

In [ ]:
#print('CV:ed TRAINING DATA: \n' + classification_report(y_train_clf, cross_val_predict(rfc1, X_train_prep, y_train_clf, cv=3)))
print('TEST DATA: \n' + classification_report(y_test_clf, rfc1.predict(X_test_prep)))

### Duplicated train/test data

In [ ]:
#All duplicates and also without alpha-days crime density columns
#Training data
dupl = X_train.duplicated().sum()
dupl_wo_alpha = X_train.drop([col for col in X_train.columns if 'alpha' in col], axis=1).duplicated().sum()
print(dupl/len(X_train)*100)
print(dupl_wo_alpha/len(X_train)*100)

In [ ]:
#Test data
dupl = X_test.duplicated().sum()
dupl_wo_alpha = X_test.drop([col for col in X_test.columns if 'alpha' in col], axis=1).duplicated().sum()
print(dupl/len(X_test)*100)
print(dupl_wo_alpha/len(X_test)*100)

### Feature importance

In [ ]:
#Using RFC attribute feature_importance_, which is based on position in decision trees (higher = more important, since it's greedy)
feature_names = list(num_mapper.transformed_names_) + list(cat_mapper.transformed_names_)
sorted(zip(rfc1.feature_importances_, feature_names), reverse=True)

### Predictions 

In [ ]:
def retrieve_imputed_assembled_feature_matrix(CDG_lookup, pdate):
    #nh_features
    nh_features = CDG_lookup.drop_duplicates('codigo_barrio')[['Area', 'Pop_cnt', 'median_age', 'median_income','Male_pop_ratio', 'Age18Plus_ratio']]

    #alpha
    alpha_cols=[col for col in CDG_lookup if 'alpha' in col]
    alpha_features = CDG_lookup.drop_duplicates(['fecha','codigo_barrio','conducta']).groupby(['codigo_barrio','conducta']).median()[alpha_cols]
    alpha_features = alpha_features.reset_index().drop('codigo_barrio', axis=1)

    #evictions
    evictions_median = CDG_lookup.drop_duplicates(['codigo_barrio','year']).median()['evictions_PkC']
    nh_year_features = pd.DataFrame(data=evictions_median, index=[0], columns={'evictions_PkC'})

    #date
    date_features = {}
    pdate = pd.to_datetime(pdate)
    date_features['weekday_num'] = pdate.weekday()
    date_features['weekend'] = np.asscalar(np.where(date_features['weekday_num'] >= 4, 1, 0))
    date_features['Season'] = (pdate - pd.DateOffset(months=1)).quarter
    date_features = pd.DataFrame(data=date_features, index=[0])

    #Crime cats
    crime_cats = pd.DataFrame(crime_data_grd.drop_duplicates('conducta')['conducta'])

    #Stitch together
    X_pred = nh_features.reset_index(drop=True).copy()
    X_pred = X_pred.merge(nh_year_features, left_index=True, right_index=True)
    X_pred = X_pred.merge(date_features, left_index=True, right_index=True)
    X_pred = X_pred.merge(crime_cats, how='right', left_index=True, right_index=True).fillna(method='ffill').fillna(method='bfill')
    X_pred = X_pred.merge(alpha_features, on='conducta')
    X_pred = X_pred.reindex(sorted(X_pred.columns), axis=1)

    return X_pred

def get_crime_probas(codigo_barrio, pdate, model=rfc1):
    #make sure we have a valid date
    try:
        pd.to_datetime(pdate)
    except:
        raise ValueError('pd.to_datetime() failed, only supports valid dates.')
    drop_columns=['Crime_count','Age18Plus_cnt', 'evictions','fecha','codigo_barrio','geometry','year']
    #Slice main table
    CDG_lookup = crime_data_grd.query('fecha == @pdate and codigo_barrio == @codigo_barrio')
    #match in lookup table, construct X_pred easily
    if len(CDG_lookup) > 0:
        X_pred = CDG_lookup.drop(drop_columns, axis=1)
        X_pred_prep = full_pipeline.transform(X_pred)
        y_pred = model.predict_proba(X_pred_prep)
    #no match in table, impute date dependent historical features, and construct X_pred
    else:
        CDG_lookup = crime_data_grd.query('codigo_barrio == @codigo_barrio')#.drop('geometry', axis=1)
        #match on codigo_barrio
        if len(CDG_lookup) > 0:
            X_pred = retrieve_imputed_assembled_feature_matrix(CDG_lookup, pdate)
            X_pred_prep = full_pipeline.transform(X_pred)
            y_pred = model.predict_proba(X_pred_prep)
        #No match on codigo_barrio, throw error
        else:
            codigo_barrios = list(crime_data_grd.drop_duplicates('codigo_barrio')['codigo_barrio'])
            raise ValueError('Only supports (correctly spelled) SF neighborhoods: \n\n'+ str(codigo_barrios))
    #print neatly
    print(f'{codigo_barrio} on {pdate}')
    for cat, pred in zip(X_pred['conducta'],np.round(y_pred[:,1],2)):
        print(f'{cat:<20}{pred:<10}')
    print('\n')
    return

In [ ]:
get_crime_probas('101', '28-02-2018')